In [1]:
#% pip install openpyxl
import os, re, configparser
import pandas as pd
import numpy as np
from globals import glob
from utilities_database import prepare_data, prepare_database, consult_database, clear_databases, retrieve_data, rename_index, get_date, get_sigma, rename_limits_table
from utilities_analysis import mean_calculator, limits_generator, ini_generator, RyR, z_score_filter, reset_df, capability_calculation
from utilities_plotting import plot_scatter, plot_capability, plot_simple_limits, plot_to_pdf
from utilities_widgets import widget_selector, widget_int_entry, widget_select_one
import ipywidgets as widgets
from ipywidgets import HBox
import PyPDF2 as pdf
import _db_tools as db
##Widgets creation
means_calculation = widget_selector(options=['Global means', 'Per position means'], description='Means algorithm', tooltips=['Calculate global means for each limit', 'Calculate means for each position'])
sigma_selector = widget_int_entry(value=6, description="Sigma value (1/2 to each tail) selected")
fiber_selection = widgets.Text(value='Guia_Luz_Blanco_FB1_X', placeholder='Type the fiber label', description='Fiber label:', disabled=False)
auto_position = widgets.Checkbox(value=True, description='Auto position', layout={'width': '200px'})
mean_slider = widgets.FloatSlider(value=0.35, min=0.3, max=0.4, step=0.005, readout_format='.3f', description='X-axis mean')
range_slider = widgets.IntSlider(value=5, min=1, max=10, step=1, description='X-axis zoom')

Database *input.db* found in: c:\Users\lucio\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\input.db
Database *output.db* found in: c:\Users\lucio\Desktop\Codebase\Python_Eiit_RyR\a2_Analyser\database\output.db


In [2]:
##File filtering
extension = "pdf"
file_list = os.listdir("../a2_output/reports")
filtered_list = [filename for filename in file_list if filename.endswith(extension)]
review_list = filtered_list[0:10] + ["..."] if len(filtered_list) > 10 else filtered_list
print("Files ready for merging:")
for index, file in enumerate(review_list):
    print(f"    {review_list[index]}")

Files ready for merging:
    VA_007_EDA_boxplot_2024y-01m-25d_18h-48m-17s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_18h-49m-45s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-09m-15s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-10m-10s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-11m-09s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-11m-21s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-13m-32s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-20m-48s.pdf
    VA_007_EDA_boxplot_2024y-01m-25d_19h-22m-56s.pdf
    VA_007_EDA_scatter_2024y-01m-25d_18h-48m-17s.pdf
    ...


In [13]:
##Data selection
tables = widgets.SelectMultiple(options=filtered_list, value=(), description='pdf files', rows=10, layout={'width': '600px'})
print("Common order: Scatter visualization, Boxplot visualization, Capability report, Capability visualization.")
print('Select the tables to load with "CTRL" or "SHIFT" pressed:')
display(tables)

Common order: Scatter visualization, Boxplot visualization, Capability report, Capability visualization.
Select the tables to load with "CTRL" or "SHIFT" pressed:


SelectMultiple(description='pdf files', layout=Layout(width='600px'), options=('VA_007_EDA_boxplot_2024y-01m-2…

In [18]:
##Data preparation
selected_files = []
for file_selected in tables.value:
    file_selected = os.path.abspath(os.path.join("../a2_output/reports", file_selected))
    selected_files.append(file_selected)

In [19]:
def merge_pdfs(input_pdfs, output_pdf):
    merger = pdf.PdfMerger()
    for file in input_pdfs:
        merger.append(file)
    with open(output_pdf, 'wb') as output_file:
        merger.write(output_file)

report_name = glob.tooling + "_final_report_" + get_date() + ".pdf"
merge_pdfs(selected_files, os.path.abspath(os.path.join("../a2_output/reports", report_name)))